In [1]:
#import libraries
import requests
import pandas as pd
import time

In [2]:
#Keys
API_KEY = "xxxxxxxxxxxxxxxxxxxxx"
CHANNEL_ID = "xxxxxxxxxxxxxxxxxxxxx"

In [6]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()
    for items in response_video_stats['items']:
        view_count = items['statistics']['viewCount']
        like_count = items['statistics']['likeCount']
        comment_count = items['statistics']['commentCount']

    return view_count, like_count, comment_count

In [14]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
#main

#build our dataframe
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"]) 

df2 = get_videos(df2)

In [12]:
df2

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,rbRYsAgwPBA,Using IFNULL() on a Google SQL Interview Question,2023-04-19,346,19,0
1,IWo8H8U_Ugk,Sorting and Ranking Data in Python: An Overvie...,2023-04-18,133,4,0
2,2uu0prXMhfU,How to Rank() Your Data In Python Pandas,2023-04-12,621,34,1
3,sF6gngs9myY,38 Data Science Tools For Beginners to Experts,2023-04-05,1675,131,12
4,Y5-eAFnxMjU,Maximizing Profits: Solving the Uber Data Proj...,2023-04-04,240,9,0
...,...,...,...,...,...,...
144,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2554,15,2
145,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1917,29,0
146,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1178,21,0
147,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8156,76,1


In [13]:
df2.to_csv('youtube_vids_1st_pull.csv')